In [ ]:
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re
from functools import reduce

In [ ]:
def similarity_cosine(text1, text2):
    X_list = word_tokenize(text1)
    Y_list = word_tokenize(text2)

    l1 = []
    l2 = []
    
    sw = stopwords.words('turkish')


    X_set = {w for w in X_list if not w in sw}
    Y_set = {w for w in Y_list if not w in sw}

    rvector = X_set.union(Y_set)
    for w in rvector:
        if w in X_set:
            l1.append(1)
        else:
            l1.append(0)
        if w in Y_set:
            l2.append(1)
        else:
            l2.append(0)
    c = 0

    for i in range(len(rvector)): #kosinüs formülü
        c += l1[i] * l2[i]
    cosine = 0

    if c > 0:
        cosine = c / float((sum(l1) * sum(l2)) ** 0.5)

    return cosine
similarity_cosine("bi̇lateral mamografi̇ i̇ncelemesi̇", "bi̇lateral mamografi̇ i̇ncelemesi̇")

In [ ]:
JSON_data = []

In [ ]:
def loadConllFile(filename):
    with open (filename, "r") as file:
        return(file.readlines())

In [ ]:
JSON_data = []
data_paths = ["json/oguzhan.jsonl", "json/serdar.jsonl", "json/ensar.json"]
for pth in data_paths:
    #JSON_data = loadConllFile("data/custom_data_train.json")
    JSON_data.extend(loadConllFile(pth))

In [ ]:
f = open("json/dataset.json", "w")
for idx, d in enumerate(JSON_data) :
    f.writelines(json.dumps(json.loads(d), ensure_ascii=False)+"\n")

In [ ]:
NEW_JSON_DATA = []
NULL_JSON_DATA = []
for i in JSON_data:
    row = json.loads(i)
    row_labels = list(row["label"])

    if len(row_labels) == 0:
        NULL_JSON_DATA.append(row)
    else:
        NEW_JSON_DATA.append(row)

In [ ]:
NEW_JSON_DATA[0]["text"]

In [ ]:
def Json2ConLL_v2(arrJsonData):
    conn_list = []
    tab = '\t'

    def check_entities(arrEnt,start_pos, end_pos):
        for ent in arrEnt:
            begin, end, e = ent
            if begin == start_pos:
                if end_pos< end:
                    return 1,None
                else:
                    return 2, e
        return 0, "O"


    for data in arrJsonData:

        jsons = data#json.loads(data) bu şimdilk eklendı, duruma gore tekrar duzenlemelı
        words = jsons["text"].split(" ")
        entities = jsons["label"]

        temp_word_start = 0
        temp_word_end = 0
        temp_word =[]
        
        for word in words:

            temp_word_end += len(word)
            
            chwcking,entt =check_entities(entities, temp_word_start, temp_word_end)
            if chwcking == 1:
                temp_word.append(word)
                temp_word_end+=1

            elif chwcking == 2:
                temp_word.append(word)
                value = " ".join(temp_word)

                conn_list.append(f"{value}{tab}{entt}") 
                temp_word_start = temp_word_end+1

                temp_word_end += 1
                temp_word.clear()
            else:
                if len(word) >0:
                    conn_list.append(f'{word}{tab}O') 
                    temp_word_start = temp_word_end+1
                    temp_word_end += 1
                    temp_word.clear()

        start_index = 0
        end_index = 0
        conn_list.append("\n")
    return conn_list

In [ ]:
data = Json2ConLL_v2(NEW_JSON_DATA)
data

In [ ]:
f = open("conll/dataset_new_case.conll", "w")
for idx, d in enumerate(data) :
    if not d in "\n":
        f.writelines(d+"\n")

    if d in "\n":
        f.writelines(".\n")
        f.writelines("\n")
        continue